In [1]:
from keras.datasets import mnist
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pdb # for debug

Using TensorFlow backend.


In [2]:
# ARIYASU code
x_ = np.array([[[[1,2,3,4]],
                [[2,3,4,5]]]]*10) # (10, 2, 1, 4)
w = np.array([[[[1,1,2]],[[2,1,1]]],
                   [[[2,1,1]],[[1,1,1]]],
                   [[[1,1,1]],[[1,1,1]]]])
b = [[[1]],[[2]],[[3]]]
loss = np.array([[[9,11]],
                [[32,35]],
                [[52,56]]])
# 確認用
out_ = np.array([[21,29],
                [18,25],
                [18,24]])
x_delta = np.array([[125,230,204,113],
                    [102,206,195,102]])
w_delta = np.array([[[31,51,71],[51,71,91]],
                    [[102,169,236],[169,236,303]],
                    [[164,272,380],[272,380,488]]])

# 2次元の畳み込みニューラルネットワークスクラッチ

2次元に対応した畳み込みニューラルネットワーク（CNN）のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


プーリング層なども作成することで、CNNの基本形を完成させます。クラスの名前はScratch2dCNNClassifierとしてください。

## データセットの用意
引き続きMNISTデータセットを使用します。2次元畳み込み層へは、28×28の状態で入力します。


今回は白黒画像ですからチャンネルは1つしかありませんが、チャンネル方向の軸は用意しておく必要があります。


`(n_samples, n_channels, height, width)` の`NCHW`または`(n_samples, height, width, n_channels)`の`NHWC`どちらかの形にしてください。



In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
X_train = X_train[:,np.newaxis, :,:, ]
X_test = X_test[:,np.newaxis, :,:, ]

# 画像サイズ縮小処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])

In [5]:
X_train.shape

(60000, 1, 28, 28)

## 【問題1】2次元畳み込み層の作成
1次元畳み込み層のクラスConv1dを発展させ、2次元畳み込み層のクラスConv2dを作成してください。


フォワードプロパゲーションの数式は以下のようになります。

$$
a_{i,j,m} = \sum_{k=0}^{K-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1}x_{(i+s),(j+t),k}w_{s,t,k,m}+b_{m}
$$

$a
_{i,j,m}$
 : 出力される配列のi行j列、mチャンネルの値


$i$
 : 配列の行方向のインデックス


$j$
 : 配列の列方向のインデックス


$m$
 : 出力チャンネルのインデックス


$K$
 : 入力チャンネル数


$F_h
,F_w$
 : 高さ方向（$h$）と幅方向（$w$）のフィルタのサイズ


$x_{(i+s),(j+t),k}$
 : 入力の配列の(i+s)行(j+t)列、kチャンネルの値


$w_{s,t,k,m}$
 : 重みの配列のs行t列目。kチャンネルの入力に対して、mチャンネルへ出力する重み


b_m
 : mチャンネルへの出力のバイアス項


全てスカラーです。


次に更新式です。1次元畳み込み層や全結合層と同じ形です。

$$
w_{s,t,k,m}^{\prime} = w_{s,t,k,m} - \alpha \frac{\partial L}{\partial w_{s,t,k,m}} \\
b_{m}^{\prime} = b_{m} - \alpha \frac{\partial L}{\partial b_{m}}
$$

$α$
  : 学習率


$\frac{\partial L}{\partial w_{s,t,k,m}} : 
w_{s,t,k,m}$ に関する損失 $L$の勾配


$\frac{\partial L}{\partial b_{m}}: b_m$
 に関する損失 
$L$
 の勾配


勾配 
$\frac{\partial L}{\partial w_{s,t,k,m}}$
 や $\frac{\partial L}{\partial b_{m}}$
 を求めるためのバックプロパゲーションの数式が以下である。
 
 $$
 \frac{\partial L}{\partial w_{s,t,k,m}} = \sum_{i=0}^{N_{out,h}-1}\sum_{j=0}^{N_{out,w}-1} \frac{\partial L}{\partial a_{i,j,m}}x_{(i+s)(j+t),k}\\
\frac{\partial L}{\partial b_{m}} = \sum_{i=0}^{N_{out,h}-1}\sum_{j=0}^{N_{out,w}-1}\frac{\partial L}{\partial a_{i,j,m}}
 $$
 
$\frac{\partial L}{\partial a_i}$: 勾配の配列のi行j列、mチャンネルの値

$N_{out,h},N_{out,w}$: 高さ方向（h）と幅方向（w）の出力のサイズ

前の層に流す誤差の数式は以下です。

$$
\frac{\partial L}{\partial x_{i,j,k}} = \sum_{m=0}^{M-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1} \frac{\partial L}{\partial a_{(i-s),(j-t),m}}w_{s,t,k,m}
$$

$\frac{\partial L}{\partial x_{i,j,k}}$: 前の層に流す誤差の配列のi列j行、kチャンネルの値

$M$
  : 出力チャンネル数
  
ただし、 
$i−s<0$
 または $i−s>N_{out,h}−1$
 または $j−t<0$
 または $j−t>N_{out,w}−1$
 のとき  
 $\frac{\partial L}{\partial a_{(i-s),(j-t),m}} =0$です。

### Conv2d

In [220]:
class Conv2d:
    """

    """
    def __init__(self, W, B):
        self.lr = 1
        self.B = np.ones(B)
        self.W = np.ones(W) # Wのシェイプ(output_ch, input_ch, FH, FW）
    
    def forward(self, x):
        """

        """
        FN, CH, FH, FW = self.W.shape
#         pdb.set_trace()
        N, CH, H, W = x.shape
        P = 0 # Padding
        S = 1 # stride

        self.x = x # for backward
        
        self.FM = np.empty((N, FN, H-FH+1, W-FW+1)) # Feature Map
        for i in range(N): # loop for n_sample times
            for j in range(FN): # loop for n_filter times
                for k in range(H-FH+1):
                    for l in range(W-FW+1):
#                         pdb.set_trace()
                        self.FM[i, j, k, l] = np.sum(x[i, :, k:k+FH, l:l+FW]*self.W[j, :, :, :]) + self.B[j]  
    
    
    
        return self.FM

    
    def backward(self, dA):
        """

        """
#         pdb.set_trace()
        FN, CH, FH, FW = self.W.shape

        dZ = np.empty(self.x.shape)
        for i in range(self.x.shape[0]): # 2→入力ch 
            for j in range(self.x.shape[1]): # N_in = ４→Xの特徴量
                for k in range(FH): # 3→w.shape[0]→出力ch
                    for l in range(FW): # Wの数 = 3
                        dZ[i, j, k:k + FH, l:l + FW] += dA[i, j, k:k + FH, l:l + FW]*self.W[i, j, :, :] 

#         pdb.set_trace() 
        self.dZ = dZ       

        # w 更新
        new_W = np.empty(self.W.shape)
        for i in range(self.F):
            new_W[i] = sum(self.X[i:i+self.N_out]*dA)
        self.W -= self.lr*new_W
        # b 更新  
        new_B = sum(dA)
        self.B -= self.lr*new_B     
    
    
        return dZ

## 【問題2】2次元畳み込み後の出力サイズ
畳み込みを行うと特徴マップのサイズが変化します。どのように変化するかは以下の数式から求められます。この計算を行う関数を作成してください。

$$
N_{h,out} =  \frac{N_{h,in}+2P_{h}-F_{h}}{S_{h}} + 1\\
N_{w,out} =  \frac{N_{w,in}+2P_{w}-F_{w}}{S_{w}} + 1
$$

$N_{out}$
  : 出力のサイズ（特徴量の数）


$N_{in}$
 : 入力のサイズ（特徴量の数）


$P$
 : ある方向へのパディングの数


$F$
 : フィルタのサイズ


$S$
 : ストライドのサイズ


$h$
 が高さ方向、 
$w$
 が幅方向である

In [208]:
def output_func(N_in, P, F, S):
    return int((N_in + 2*P - F)/S + 1)
    
Fh = 3
Fw = 3
Sh = 1
Sw = 1
Ph = 0
Pw = 0

Nh_in = 28
Nw_in = 28

Nh_out = output_func(Nh_in, Ph, Fh, Sh)
Nw_out = output_func(Nw_in, Pw, Fw, Sw)

Nh_out, Nw_out

(26, 26)

## 【問題3】最大プーリング層の作成
最大プーリング層のクラスMaxPool2Dを作成してください。プーリング層は数式で表さない方が分かりやすい部分もありますが、数式で表すとフォワードプロパゲーションは以下のようになります。
$$
a_{i,j,k} = \max_{(p,q)\in P_{i,j}}x_{p,q,k}
$$

$P_{i,j}$
 : $i$行$j$列への出力する場合の入力配列のインデックスの集合。 

$S_h
×
S_w$
 の範囲内の行（$p$）と列（$q$）


$S_h,S_w$
 : 高さ方向（$h$）と幅方向（$w$）のストライドのサイズ


$(p,q)∈P_{i,j}: P_{i,j}$
 に含まれる行（$p$）と列（$q$）のインデックス


$a_{i,j,m}$
 : 出力される配列の$i$行$j$列、$k$チャンネルの値


$x_{p,q,k}$
 : 入力の配列の$p$行$q$列、$k$チャンネルの値


ある範囲の中でチャンネル方向の軸は残したまま最大値を計算することになります。


バックプロパゲーションのためには、フォワードプロパゲーションのときの最大値のインデックス 
$(p,q)$
 を保持しておく必要があります。フォワード時に最大値を持っていた箇所にそのままの誤差を流し、そこ以外には0を入れるためです。



### MaxPool2D

In [221]:
#入力はNHWC(n_samples, height, width, n_channels)
class MaxPool2D:
    
    def __init__(self):
        self.FH = 2
        self.FW = 2
        self.S = 1 # stride
        
    def forward(self, x):
        self.A = np.zeros(x.shape)
        
        
        sample = x.shape[0]
        chanel = x.shape[1]
        height = x.shape[2]
        width = x.shape[3]
        self.Z_index = np.zeros([sample,chanel,height,width])
        Z = np.zeros([sample,chanel,int(height/2),int(width/2)])

        for h in range(sample):#サンプルを設定
            for k in range(chanel):#チャネルを設定
                for i in range(self.FH):#高さを設定 フィルタ＝ストライド＝２
                    for j in range(self.FW):#幅を設定
                            Z0 = x[h,k,i*2:i*2 + 2,j*2:j*2 + 2]
                            Z[h,k,i,j] = np.nanmax(Z0)
                            a = Z0/np.nanmax(Z0)#最大値が１の行列
                            self.Z_index[h,k,i*2:i*2 + 2,j*2:j*2 + 2] += (np.where(a == 1 ,1,0)) # z idx keep
        return Z
    
    def backward(self, x):
        return self.A
        

## 【問題5】平滑化
平滑化するためのFlattenクラスを作成してください。


フォワードのときはチャンネル、高さ、幅の3次元を1次元にreshapeします。その値は記録しておき、バックワードのときに再びreshapeによって形を戻します。


この平滑化のクラスを挟むことで出力前の全結合層に適した配列を作ることができます。

### Flatten

In [210]:
class Flatten:
    
    def __init__(self):
        pass
    
    def forward(self, x):
        self.flatten = x.shape
        self.FLT_shape = x.reshape(x.shape[0], -1)[1]
        return x.reshape(x.shape[0], -1)
    
    def backward(self, x):
        return x.reshape(self.flatten)
        
    

# 検証

## 【問題6】学習と推定
作成したConv2dを使用してMNISTを学習・推定し、Accuracyを計算してください。


精度は低くともまずは動くことを目指してください。

### Initializer

#### SimpleInitializer

In [211]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)       
        return W
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(n_nodes2)
        return B

#### Xavier

In [212]:
class Xavier:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      Xavierの初期値
    """
    def __init__(self, n_nodes1):
        self.sigma = 1/np.sqrt(n_nodes1) # self.batch_sizeかも
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)       
        return W
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(n_nodes2)
        return B

#### He

In [213]:
class He:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      Xavierの初期値
    """
    def __init__(self, n_nodes1):
        self.sigma = np.sqrt(2/n_nodes1) 
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)       
        return W
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(n_nodes2)
        return B

### Optimizer

In [214]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        # layerの引数は"class FCのインスタンス自体"を取得している。

        layer.B -= self.lr*layer.B_dash # B更新
        layer.W -= self.lr*layer.W_dash # W更新


### FC

In [215]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes_out, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
#         self.n_nodes1 = n_nodes1
        self.n_nodes_out = n_nodes_out
        self.initializer = initializer
        self.optimizer = optimizer
        self.H_W = 0
        self.H_B = 0
        
    def forward(self, x, n_nodes_in):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        
        self.n_nodes_in = n_nodes_in
        self.W = self.initializer.W(self.n_nodes_in, self.n_nodes_out)
        self.B = self.initializer.B(self.n_nodes_out)
        self.Z = x # 前のレイヤーのZを取得
        A = x@self.W + self.B
        
        return A

    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
#         pdb.set_trace()
        dZ = dA@self.W.T # このレイヤーのW

        # 更新
        self.B_dash = np.mean(dA, axis=0)
        self.W_dash = self.Z.T@dA # 前のレイヤーのZ
        self = self.optimizer.update(self) # この引数のselfはこの"class FCのインスタンス自体"を取得している。
        return dZ

### 活性化関数

In [216]:
class ReLU:

    def __init__(self):
        pass
    
    def forward(self, x):
        self.F = np.where(x > 0, 1, 0) # backwardの引数のためのAを保管
        return np.maximum(x, 0)
    
    def backward(self, x):
#         pdb.set_trace()
        return x*self.F
    

class Softmax:
    def __init__(self):
        pass
    def cross_entropy_func(self, x, y):
        li = np.empty(len(x))
        for i in range(len(x)):
            lj = np.empty(10) # 10 = self.n_output, 最終的に変数にする
            for j in range(10): # 10 = self.n_output, 最終的に変数にする
                lj[j] = y[i,j]*np.log(x[i,j])
            li[i] = sum(lj)
        self.l = sum(li)/-10 # 10 = self.n_output, 最終的に変数にする
    
    def forward(self, x):
        x_max = np.max(x, axis=1)
        exp_x = np.exp(x - x_max.reshape(-1, 1))
        sum_exp_x = np.sum(exp_x, axis=1).reshape(-1, 1)  
        return exp_x/sum_exp_x

    def backward(self, x, y):
        self.cross_entropy_func(x, y)
        return x - y

In [217]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=42):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

### ScratchCNNClassifier

In [218]:
class ScratchCNNClassifier:

    def __init__(self, verbose=None, batch_size=20, a=0.01, n_epoch=5):
        self.verbose = verbose
        self.batch_size = batch_size
        self.lr = a
        self.n_epoch = n_epoch


    def fit(self, X, y, X_val=None, y_val=None):
# 初期値
        NODE_TO_SOFTMAX = 10 # softmaxに送るノード数（固定）
        self.sigma = 0.01 # ガウス分布の標準偏差

# X設定　（60000, 1, 28, 28）の場合想定        
#         self.n_samples, self.input_ch, self.height, self.width = X.shape        
        self.n_samples = X.shape[0]        
        self.input_ch = X.shape[1]        
        self.height = X.shape[2]        
        self.width = X.shape[3]        
        
# Filter
        self.W_1 =32, X.shape[1], 3, 3 # Wのシェイプ(output_ch, input_ch, FH, FW）
# Bias
        self.B_1 = self.W_1[0] # Bのシェイプ（output_ch）
        
# fitメソッド内        
        self.Conv2d1 = Conv2d(self.W_1, self.B_1)
        self.activation1 = ReLU()
        self.MaxPool2D_1 = MaxPool2D()
        self.Flatten = Flatten()
        optimizer = SGD(self.lr)
        self.FC1 = FC(NODE_TO_SOFTMAX, SimpleInitializer(self.sigma), optimizer)
        self.SoftMax = Softmax()
        
        get_mini_batch = GetMiniBatch(X, y, batch_size = self.batch_size)
        self.l_list = np.empty(self.n_epoch)
        self.l_val_list = np.empty(self.n_epoch)
        self.cnt = 0
        self.cnt_list = []
        for i in range(self.n_epoch):
            for X, y in get_mini_batch:        
        
        
# イテレーションごとのフォワード  
                # Input Image (28, 28, 1)
                FM1 = self.Conv2d1.forward(X) # (26, 26, 32)
                Z1 = self.activation1.forward(FM1)
                FM2 = self.MaxPool2D_1.forward(Z1) # (13, 13, 32)
                FLT = self.Flatten.forward(FM2) # (128, 1)
                n_node_out = FLT.shape[1]
                A1 = self.FC1.forward(FLT, n_node_out) # (64, 1) 
                Z2 = self.SoftMax.forward(A1) # (10, 1)              
#                 pdb.set_trace()
# イテレーションごとのバックワード
                dA1 = self.SoftMax.backward(Z2, y) # 交差エントロピー誤差とソフトマックスを合わせている
#                 pdb.set_trace()
                dFLT = self.FC1.backward(dA1) # 交差エントロピー誤差とソフトマックスを合わせている
                dFM2 = self.Flatten.backward(dFLT)
                dZ1 = self.MaxPool2D_1.backward(dFM2) # 
                dFM1 = self.activation1.backward(dZ1)
                dZ0 = self.Conv2d1.backward(dFM1) # dZ0は使用しない        
                


    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        A1 = self.Conv2d1.forward(X) # (26, 26, 32)
        A2 = self.MaxPool2D_1.forward(A1) # (13, 13, 32)
        A3 = self.Conv2d12.forward(A2) # (11, 11, 64)
        A4 = self.MaxPool2D_2.forward(A3) # (5, 5, 64)
        A5 = self.Conv2d13.forward(A4) # (3, 3, 128)
        A6 = self.MaxPool2D_3.forward(A5) # (1, 1, 128)
        A7 = self.Flatten.forward(A6) # (128, 1)
        A8 = self.FC.forward(A7) # (64, 1) 
        A9 = self.SoftMax.forward(A8) # (10, 1)   
        
        return np.argmax(A9, axis=1)

In [219]:
model = ScratchCNNClassifier()
model.fit(X_train, y_train, X_test, y_test)

AttributeError: 'Conv2d' object has no attribute 'F'

## 【問題9】出力サイズとパラメータ数の計算
CNNモデルを構築する際には、全結合層に入力する段階で特徴量がいくつになっているかを事前に計算する必要があります。


また、巨大なモデルを扱うようになると、メモリや計算速度の関係でパラメータ数の計算は必須になってきます。フレームワークでは各層のパラメータ数を表示させることが可能ですが、意味を理解していなくては適切な調整が行えません。


以下の3つの畳み込み層の出力サイズとパラメータ数を計算してください。パラメータ数についてはバイアス項も考えてください。


1.


* 入力サイズ : 144×144, 3チャンネル
* フィルタサイズ : 3×3, 6チャンネル
* ストライド : 1
* パディング : なし

2.


* 入力サイズ : 60×60, 24チャンネル
* フィルタサイズ : 3×3, 48チャンネル
* ストライド　: 1
* パディング : なし

3.


* 入力サイズ : 20×20, 10チャンネル
* フィルタサイズ: 3×3, 20チャンネル
* ストライド : 2
* パディング : なし

＊最後の例は丁度良く畳み込みをすることができない場合です。フレームワークでは余ったピクセルを見ないという処理が行われることがあるので、その場合を考えて計算してください。端が欠けてしまうので、こういった設定は好ましくないという例です。